## Debugging With Git Bisect

You can use

``` bash
git bisect
```

to find out which commit caused a bug.

### An example repository

In a nice open source example, I found an arbitrary exemplar on github

In [1]:
import os
top_dir = os.getcwd()
git_dir = os.path.join(top_dir, 'learning_git')
os.chdir(git_dir)

In [2]:
%%bash
rm -rf bisectdemo
git clone https://github.com/UCL-RITS/bisectdemo.git

Cloning into 'bisectdemo'...


In [3]:
bisect_dir=os.path.join(git_dir,'bisectdemo')
os.chdir(bisect_dir)

In [4]:
%%bash
python squares.py 2 # 4

4


This has been set up to break itself at a random commit, and leave you to use
bisect to work out where it has broken:

In [5]:
%%bash
./breakme.sh > break_output

Switched to a new branch 'buggy'


Which will make a bunch of commits, of which one is broken, and leave you in the broken final state

In [6]:
%%bash
python squares.py 2 # Error message

Traceback (most recent call last):
  File "squares.py", line 9, in <module>
    print(integer**2)
TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'


CalledProcessError: Command 'b'python squares.py 2 #\xc2\xa0Error message\n'' returned non-zero exit status 1.

### Bisecting manually

In [10]:
%%bash
git bisect start
git bisect bad # We know the current state is broken
git switch main
git bisect good # We know the main branch state is OK

Your branch is up-to-date with 'origin/master'.
Bisecting: 500 revisions left to test after this (roughly 9 steps)
[f85561dbe54b69fc7f037e3fd1b742b7395726b5] Comment 500


Previous HEAD position was f85561d... Comment 500
Switched to branch 'buggy'
Switched to branch 'master'


Bisect needs one known good and one known bad commit to get started

### Solving Manually

``` bash
python squares.py 2 # 4
git bisect good
python squares.py 2 # 4
git bisect good
python squares.py 2 # 4
git bisect good
python squares.py 2 # Crash
git bisect bad
python squares.py 2 # Crash
git bisect bad
python squares.py 2 # Crash
git bisect bad
python squares.py 2 #Crash
git bisect bad
python squares.py 2 # 4
git bisect good
python squares.py 2 # 4
git bisect good
python squares.py 2 # 4
git bisect good
```


And eventually:

``` bash
git bisect good
    Bisecting: 0 revisions left to test after this (roughly 0 steps)

python squares.py 2
    4

git bisect good
2777975a2334c2396ccb9faf98ab149824ec465b is the first bad commit
commit 2777975a2334c2396ccb9faf98ab149824ec465b
Author: Shawn Siefkas <shawn.siefkas@meredith.com>
Date:   Thu Nov 14 09:23:55 2013 -0600

    Breaking argument type

```

Stop the bisect process with:

``` bash
git bisect reset
```

### Solving automatically

If we have an appropriate unit test, we can do all this automatically:

(*NOTE*: You don't need [to redirect the `stderr` and `stdout`](https://linuxize.com/post/bash-redirect-stderr-stdout/) (with `&>`) of `git bisect run` to a file when running these commands outside a jupyter notebook (i.e., on a shell). This is done here so the errors appears with the right commits)

In [11]:
%%bash
git bisect start
git bisect bad HEAD # We know the current state is broken
git bisect good main # We know main is good
git bisect run python squares.py 2 &> gitbisect.out
cat gitbisect.out

Bisecting: 500 revisions left to test after this (roughly 9 steps)
[8ab337c43ca34bf91ab17cc55890be57d3567c82] Comment 500
running python squares.py 2
4
Bisecting: 250 revisions left to test after this (roughly 8 steps)
[d25fb2ac5a0ed2e9474bec5a32e52eaf6f96ff6c] Comment 749
running python squares.py 2
Traceback (most recent call last):
  File "squares.py", line 9, in <module>
    print(integer**2)
TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'
Bisecting: 124 revisions left to test after this (roughly 7 steps)
[cc0d01b552be8314dc9734c78b3363b620cb325c] Comment 624
running python squares.py 2
Traceback (most recent call last):
  File "squares.py", line 9, in <module>
    print(integer**2)
TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'
Bisecting: 62 revisions left to test after this (roughly 6 steps)
[4b46a6e37c388d260d61409e51864ae77ba19d6d] Comment 562
running python squares.py 2
4
Bisecting: 31 revisions left to test after this (roughly 

Previous HEAD position was 1694ac7 Comment 592
Switched to branch 'buggy'


Boom!